In [8]:
!pip install lime # Install the missing 'lime' library

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import shap
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt
import joblib
import json

# -----------------------
# Configuration
# -----------------------
DATA_PATH = "Telco-Customer-Churn.csv"  # change if necessary
RANDOM_STATE = 42
OUTPUT_DIR = "churn_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# -----------------------
# Load data
# -----------------------
df = pd.read_csv('Telco_customer_churn.csv')
print("Loaded data shape:", df.shape)
print("DataFrame columns:", df.columns.tolist()) # Added to inspect column names

# -----------------------
# Basic cleaning for typical Telco dataset quirks
# -----------------------
# Some versions store TotalCharges as string with blanks for 0-tenure rows
if "Total Charges" in df.columns: # Corrected column name
    df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce') # Corrected column name
    df['Total Charges'] = df['Total Charges'].fillna(0.0) # Corrected column name

# Target encoding
if 'Churn Label' in df.columns: # Changed 'Churn' to 'Churn Label'
    df['ChurnFlag'] = df['Churn Label'].map({'Yes': 1, 'No': 0}) # Changed 'Churn' to 'Churn Label'
    target = 'ChurnFlag'
else:
    raise ValueError("Expecting a column named 'Churn Label' with values 'Yes'/'No'.") # Changed 'Churn' to 'Churn Label'

# -----------------------
# Feature Engineering - at least 3 domain-relevant features
# -----------------------
# 1) num_services: count of paid services (PhoneService, multiple internet-related services)
service_cols = [
    'Phone Service', 'Multiple Lines', 'Online Security', 'Online Backup',
    'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies'
]
# Ensure columns exist; if not, create placeholders
for c in service_cols:
    if c not in df.columns:
        df[c] = 'No'

# binary indicator for internet service presence
df['HasInternet'] = df['Internet Service'].apply(lambda x: 0 if str(x).lower() in ['no', 'none', 'nan'] else 1)

# Count 'Yes' across service columns (treat 'No phone service' / 'No internet service' as No)
def is_yes(x):
    if pd.isna(x): return 0
    x = str(x).lower()
    return 1 if x.startswith('yes') else 0

df['num_services'] = df[service_cols].applymap(is_yes).sum(axis=1) + df['HasInternet']

# 2) tenure_bin: bucketize tenure
def tenure_bucket(t):
    if t <= 6: return '0-6'
    if t <= 12: return '7-12'
    if t <= 24: return '13-24'
    if t <= 48: return '25-48'
    if t <= 72: return '49-72'
    return '72+'
df['Tenure Months'] = df['Tenure Months'].astype(int) # Change column 'tenure' to 'Tenure Months'
df['tenure_bin'] = df['Tenure Months'].apply(tenure_bucket) # Change column 'tenure' to 'Tenure Months'

# 3) avg_charge_per_service: MonthlyCharges / max(1, num_services) to represent value per service
df['avg_charge_per_service'] = df.apply(lambda r: r['Monthly Charges'] / max(1, r['num_services']), axis=1) # Corrected column name

# 4) high_autopay_flag: PaymentMethod containing 'automatic' or 'autopay' -> often 'Electronic check' vs 'Bank transfer (automatic)'
df['AutoPay'] = df['Payment Method'].astype(str).str.contains('automatic|auto|bank transfer|credit card', case=False, na=False).astype(int)

# 5) contract_months_est: heuristically map Contract to expected months remaining (Ordinal)
contract_map = {'Month-to-month': 1, 'One year': 12, 'Two year': 24}
df['contract_term'] = df['Contract'].map(contract_map).fillna(1)

# Additional small cleaning (drop customerID)
if 'CustomerID' in df.columns: # Changed 'customerID' to 'CustomerID'
    df = df.drop(columns=['CustomerID']) # Changed 'customerID' to 'CustomerID'

# -----------------------
# Feature list
# -----------------------
# Select features: include numeric + engineered + some categorical
num_features = ['Tenure Months', 'Monthly Charges', 'Total Charges', 'num_services', 'avg_charge_per_service', 'contract_term'] # Corrected column names
cat_features = ['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service',
                'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup',
                'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies',
                'Contract', 'Paperless Billing', 'Payment Method', 'tenure_bin'] # Corrected column names

# Ensure all categorical fields exist
cat_features = [c for c in cat_features if c in df.columns]

features = num_features + cat_features

# Keep only selected columns + target
df_model = df[features + [target]].copy()

# Handle missing values: numeric -> median, categorical -> 'Missing'
for c in num_features:
    if df_model[c].isna().sum() > 0:
        df_model[c] = df_model[c].fillna(df_model[c].median())
for c in cat_features:
    df_model[c] = df_model[c].fillna('Missing')

# -----------------------
# Train/test split (stratified)
# -----------------------
X = df_model[features]
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=RANDOM_STATE
)
print("Train/test shapes:", X_train.shape, X_test.shape)

# -----------------------
# Preprocessing pipeline
# -----------------------
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Use OneHotEncoder for categorical (drop='rare' not available; use handle_unknown)
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ],
    remainder='drop'
)

# -----------------------
# Baseline model and tuning with RandomizedSearchCV
# -----------------------
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=RANDOM_STATE,
    n_jobs=4
)

pipe = Pipeline(steps=[('pre', preprocessor), ('clf', xgb)])

# Quick baseline fit (to report baseline metrics before heavy tuning)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]
baseline_auc = roc_auc_score(y_test, y_proba)
baseline_f1 = f1_score(y_test, y_pred)
print(f"Baseline AUC: {baseline_auc:.4f}, Baseline F1: {baseline_f1:.4f}")

# Hyperparameter search space for XGBoost (randomized)
param_dist = {
    'clf__n_estimators': [100, 200, 400],
    'clf__max_depth': [3, 4, 5, 6],
    'clf__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'clf__subsample': [0.6, 0.8, 1.0],
    'clf__colsample_bytree': [0.5, 0.7, 1.0],
    'clf__reg_alpha': [0, 0.001, 0.01, 0.1],
    'clf__reg_lambda': [1, 5, 10]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
rs = RandomizedSearchCV(pipe, param_dist, n_iter=25, scoring='roc_auc', n_jobs=4, cv=cv, random_state=RANDOM_STATE, verbose=1)
rs.fit(X_train, y_train)

print("Best params:", rs.best_params_)
best_model = rs.best_estimator_

# Save model
joblib.dump(best_model, os.path.join(OUTPUT_DIR, "xgb_churn_model.joblib"))

# -----------------------
# Evaluate final model
# -----------------------
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
print(f"Final AUC: {auc:.4f}, Final F1: {f1:.4f}")
print(classification_report(y_test, y_pred))

# Save basic metrics to JSON
metrics = {"baseline_auc": float(baseline_auc), "baseline_f1": float(baseline_f1),
           "final_auc": float(auc), "final_f1": float(f1)}
with open(os.path.join(OUTPUT_DIR, "metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

# -----------------------
# SHAP Global Interpretation
# -----------------------
# We need the trained model and the preprocessor to produce shap values.
# Extract transformed training data to pass to SHAP
pre = best_model.named_steps['pre']
clf = best_model.named_steps['clf']

# Build feature names after OneHotEncoding
# get numeric names
numeric_names = num_features
# get categorical feature names from onehot encoder
ohe = pre.named_transformers_['cat'].named_steps['onehot']
ohe_feature_names = []
if hasattr(ohe, 'get_feature_names_out'):
    ohe_feature_names = list(ohe.get_feature_names_out(cat_features))
else:
    # fallback
    for i, col in enumerate(cat_features):
        # we cannot easily infer categories without get_feature_names_out
        ohe_feature_names.append(col)

feature_names = numeric_names + ohe_feature_names

# transform a sample of training data
X_train_trans = pre.transform(X_train)
X_test_trans = pre.transform(X_test)

# SHAP TreeExplainer for XGBoost
explainer = shap.TreeExplainer(clf)
# compute SHAP values (might be heavy);
shap_vals = explainer.shap_values(X_test_trans)  # for XGBClassifier this returns array of shape (n_samples, n_features)
# shap expects numpy arrays
shap_summary_vals = shap_vals if isinstance(shap_vals, np.ndarray) else np.array(shap_vals)

# Create a DataFrame of mean absolute SHAP values per feature
mean_abs_shap = np.abs(shap_summary_vals).mean(axis=0)
shap_imp_df = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap
}).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

top5_shap = shap_imp_df.head(25)  # top 25 for context, but we'll extract top5 later
top5 = shap_imp_df.head(5)
print("Top 5 features by mean(|SHAP|):")
print(top5)

# Save SHAP summary plot
plt.figure(figsize=(10, 8))
# Use SHAP's summary_plot (saves to file via plt.savefig)
shap.summary_plot(shap_summary_vals, features=X_test_trans, feature_names=feature_names, show=False)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "shap_summary.png"), dpi=150)
plt.close()

# Save shap importance to CSV
shap_imp_df.to_csv(os.path.join(OUTPUT_DIR, "shap_feature_importance.csv"), index=False)

# -----------------------
# Local explanations using LIME
# -----------------------
# We'll pick five representative customers:
# - 2 likely churners: top predicted probability customers from test set
# - 2 likely non-churners: bottom predicted probability from test set
# - 1 ambiguous: nearest to 0.5 predicted probability

X_test_reset = X_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)
probs = best_model.predict_proba(X_test_reset)[:, 1]
preds = best_model.predict(X_test_reset)

X_test_reset['pred_proba'] = probs
X_test_reset['pred'] = preds
X_test_reset['true'] = y_test_reset.values

# Choose indexes
idx_top = X_test_reset.sort_values('pred_proba', ascending=False).head(2).index.tolist()
idx_bottom = X_test_reset.sort_values('pred_proba', ascending=True).head(2).index.tolist()
# ambiguous: closest to 0.5
idx_amb = (X_test_reset['pred_proba'] - 0.5).abs().sort_values().head(1).index.tolist()
selected_idx = idx_top + idx_bottom + idx_amb
selected_idx = list(dict.fromkeys(selected_idx))  # unique

print("Selected LIME indices:", selected_idx)

# For LIME we need the raw X_train (numpy) and feature names in original order
# Build an array of the original features (not preprocessed) for the LimeTabularExplainer
X_train_raw = X_train.copy()
X_test_raw = X_test_reset.copy()
# Note: LimeTabularExplainer wants a numpy array and class_names
class_names = ['NoChurn', 'Churn']

# We'll provide a prediction function that accepts raw rows and outputs probability for class 1
def predict_proba_raw(X_raw):
    # X_raw is a numpy array with columns corresponding to features
    X_df = pd.DataFrame(X_raw, columns=X_train_raw.columns)
    return best_model.predict_proba(X_df.values)

# But LimeTabularExplainer typically expects X_train as numeric matrix.
# We'll build an explainer using preprocessed data but also keep mapping back to original feature names.
explainer = LimeTabularExplainer(
    training_data=pre.transform(X_train_raw),
    feature_names=feature_names,
    class_names=class_names,
    mode='classification',
    random_state=RANDOM_STATE
)

local_explanations = {}

for idx in selected_idx:
    row_raw = X_test_raw.loc[idx:idx, X_train_raw.columns]
    row_trans = pre.transform(row_raw)
    exp = explainer.explain_instance(row_trans[0], clf.predict_proba, num_features=10)
    # Save HTML explanation
    html_path = os.path.join(OUTPUT_DIR, f"lime_explanation_idx_{idx}.html")
    with open(html_path, "w") as f:
        f.write(exp.as_html())
    # Extract explanation list (feature, weight)
    exp_list = exp.as_list()
    local_explanations[int(idx)] = {
        "index": int(idx),
        "pred_proba": float(X_test_reset.loc[idx, 'pred_proba']),
        "pred": int(X_test_reset.loc[idx, 'pred']),
        "true": int(X_test_reset.loc[idx, 'true']),
        "explanation": exp_list
    }

# Save local explanations JSON
with open(os.path.join(OUTPUT_DIR, "local_explanations.json"), "w") as f:
    json.dump(local_explanations, f, indent=2)

# -----------------------
# Generate plain-text reports
# -----------------------
# 1) Global Interpretation Summary (top 5 features)
global_report = []
global_report.append("Global Interpretation Summary (SHAP): Top features driving churn\n")
for i, row in top5.iterrows():
    fname = row['feature']
    val = row['mean_abs_shap']
    global_report.append(f"{i+1}. {fname} — mean(|SHAP|) = {val:.5f}")

global_report_text = "\n".join(global_report)
with open(os.path.join(OUTPUT_DIR, "global_shap_summary.txt"), "w") as f:
    f.write(global_report_text)

# 2) Local Explanation Summaries (for the selected 5 customers)
local_text_lines = []
local_text_lines.append("Local Explanations (LIME) for selected customers\n")
for idx, info in local_explanations.items():
    local_text_lines.append(f"Index {idx}: Pred_Prob={info['pred_proba']:.3f} Pred={info['pred']} True={info['true']}")
    for feat, weight in info['explanation']:
        local_text_lines.append(f"  {feat} -> weight {weight:.4f}")
    local_text_lines.append("")

local_text = "\n".join(local_text_lines)
with open(os.path.join(OUTPUT_DIR, "local_lime_summary.txt"), "w") as f:
    f.write(local_text)

# 3) Actionable Strategies (synthesis) - simple heuristic mapping
actionable = [
    "1. Target month-to-month customers with high MonthlyCharges and low tenure with discounted multi-month retention offers.",
    "2. For customers with high avg_charge_per_service but low number of services, offer bundle discounts and highlight value (upsell protective services).",
    "3. For customers with payment friction (Electronic check) and high churn risk, encourage AutoPay adoption with small incentive and clearer billing communication.",
    "4. Identify high-risk senior customers with few digital services for personalized retention calls and loyalty packages."
]
with open(os.path.join(OUTPUT_DIR, "actionable_strategies.txt"), "w") as f:
    f.write("\n".join(actionable))

# Print where outputs are saved
print(f"Outputs saved to folder: {OUTPUT_DIR}")
print("Files:")
for fn in os.listdir(OUTPUT_DIR):
    print("-", fn)

# Print brief summary to console
print("\n=== Summary ===")
print("Final AUC:", auc)
print("Final F1:", f1)
print("\nTop 5 SHAP features:")
print(top5.to_string(index=False))
print("\nSelected LIME indices and their pred_proba:")
for idx in selected_idx:
    print(f"- idx {idx}: prob {X_test_reset.loc[idx, 'pred_proba']:.3f}, pred {X_test_reset.loc[idx,'pred']}, true {X_test_reset.loc[idx,'true']}")

Loaded data shape: (7043, 33)
DataFrame columns: ['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value', 'Churn Score', 'CLTV', 'Churn Reason']
Train/test shapes: (5634, 23) (1409, 23)
Baseline AUC: 0.8316, Baseline F1: 0.5820
Fitting 3 folds for each of 25 candidates, totalling 75 fits
Best params: {'clf__subsample': 0.6, 'clf__reg_lambda': 10, 'clf__reg_alpha': 0, 'clf__n_estimators': 200, 'clf__max_depth': 3, 'clf__learning_rate': 0.05, 'clf__colsample_bytree': 0.5}
Final AUC: 0.8589, Final F1: 0.5965
              precision    recall  f1-score   support

           0       0.85      0.90      